In [9]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import re
import threading
import pandas as pd

# New Cars Scraping

In [35]:
New_Cars_For_Sale = {'Model':[], 'Model_1': [], 'Model Year': [],'Car Status': [],'Car Price': [],'Body Shape': [],'Transmission': [],'Fuel Type': [],'Engine Capacity': [],"Car About":[],'Contact':[]}

get item Attributes By xpath for each item

In [11]:
from selenium.common.exceptions import NoSuchElementException

def find_element_or_none(driver, xpath):
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        return None

lock = threading.Lock()

def get_item_info(driver, url):
    # Existing code for retrieving item info...
    # ...

    # All info
    a = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div/h1/span[1]")

    b = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div/h1/span[2]")

    c = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div/h1/span[3]")

    d = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div/h1/span[4]")

    e = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[2]")

    f = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[2]/app-spec-item/p[1]")

    g = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[3]/app-spec-item/p[1]")

    h = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[4]/app-spec-item/p[1]")

    i = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[5]/app-spec-item/p[1]")

    j = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div")

    # Handle the contact button
    k = None
    con =None
    try:
        k = driver.find_element(By.XPATH, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[3]/a")
        con = k.get_attribute('href')
    except NoSuchElementException:
        try:
            k = driver.find_element(By.XPATH, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[4]/a")
            con = k.get_attribute('href')
        except NoSuchElementException:
            print("Contact button not found.")
            con = None
                
    with lock:
        New_Cars_For_Sale['Model'].append(a)
        New_Cars_For_Sale['Model_1'].append(b)
        New_Cars_For_Sale['Model Year'].append(c)
        New_Cars_For_Sale['Car Status'].append(d)
        New_Cars_For_Sale['Car Price'].append(e)
        New_Cars_For_Sale['Body Shape'].append(f)
        New_Cars_For_Sale['Transmission'].append(g)
        New_Cars_For_Sale['Fuel Type'].append(h)
        New_Cars_For_Sale['Engine Capacity'].append(i)
        New_Cars_For_Sale['Car About'].append(j)    
        New_Cars_For_Sale['Contact'].append(con)

Get links to all items on the site

In [12]:
def new_scrap_by_src(src):
    driver = webdriver.Chrome()
    driver.get(src)
    wait = WebDriverWait(driver, 10)  # Increase the wait time here (e.g., 10 seconds)Get links to all items on the site
     
    # Scroll to the bottom of the page to load all elements
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height 

    item_links = []
    # //*[@class="n-engine-card__img"]
    Items_List = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[contains(concat(' ', normalize-space(@class), ' '), ' d-block ') and contains(concat(' ', normalize-space(@class), ' '), ' bg--white ') and contains(concat(' ', normalize-space(@class), ' '), ' car-link ')]")))
    for item in Items_List:
        item_links.append(item.get_attribute('href'))
    print(f"Loading {len(item_links)} items..")
    #############
    for i in item_links:
        time.sleep(1)
        driver.refresh()
        retries = 3
        for attempt in range(retries):
            try:
                driver.get(i)   
                get_item_info(driver, i)
                break
            except Exception as e:
                time.sleep(1)
                print(f"Failed attempt {attempt+1}/{retries}")    
    
    #############
    driver.quit()


get a number of site pages

In [28]:
def get_number_of_pages(url, xpath):
    driver = webdriver.Chrome()
    driver.get(url)

    # Scroll to the bottom of the page to load all elements
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height 

    page_bar_element = driver.find_element(By.XPATH,xpath)
    num_pages = page_bar_element.text.strip()
    driver.quit()

    return num_pages

In [6]:
url = "https://www.contactcars.com/en/newcars4sale?page=1&sortOrder=false&sortBy=createdAt"  # Replace this with the URL of the website you want to scrape
xpath = '/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/div[2]/div/div[2]/div[2]/app-new-paginator/div'
p = get_number_of_pages(url, xpath)
pages = int(re.findall(r'\d+', p)[-1])
pages

31

In [ ]:
for i in range(1,pages+ 1) :
    url = f'https://www.contactcars.com/en/newcars4sale?page={i}&sortOrder=false&sortBy=createdAt'
    new_scrap_by_src(url)
    print(f"(new cars) -Page:  {i} Loaded Successfully")

In [4]:
def NEW_Cars_Thread(u_p):
    pages = u_p
    import threading
    num_threads = 4
    # Define the function for each thread
    def scrape_thread(page_range):
        for i in page_range:
            url = f'https://www.contactcars.com/en/newcars4sale?page={i}&sortOrder=false&sortBy=createdAt'
            new_scrap_by_src(url)
            print(f"(new cars) -Page:  {i} Loaded Successfully")

    # Create and start the threads
    threads = []
    pages_per_thread = pages // num_threads  # Integer division to split pages evenly

    for t in range(num_threads):
        start_page = t * pages_per_thread + 1
        end_page = (t + 1) * pages_per_thread + 1
        if t == num_threads - 1:  # Handle remaining pages for the last thread
            end_page = pages + 1
        page_range = range(start_page, end_page)
        thread = threading.Thread(target=scrape_thread, args=(page_range,))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()
    print(threads)

In [5]:
#NEW_Cars_Thread(pages)

In [14]:
import pandas as pd
New_Cars = pd.DataFrame(New_Cars_For_Sale)
len(New_Cars)

0

In [ ]:
print(len(New_Cars_For_Sale['Model']))
print(len(New_Cars_For_Sale['Model_1']))
print(len(New_Cars_For_Sale['Model Year']))
print(len(New_Cars_For_Sale['Car Status']))
print(len(New_Cars_For_Sale['Car Price']))
print(len(New_Cars_For_Sale['Body Shape']))
print(len(New_Cars_For_Sale['Transmission']))
print(len(New_Cars_For_Sale['Fuel Type']))
print(len(New_Cars_For_Sale['Engine Capacity']))
print(len(New_Cars_For_Sale['Car About']))
print(len(New_Cars_For_Sale['Contact']))

In [ ]:
New_Cars['Location'] = New_Cars['Car About'].apply(lambda x: x.split('\n')[-2] if x else '')
New_Cars['Publishing Date'] = New_Cars['Car About'].apply(lambda x: x.split('\n')[-1] if x else '')
New_Cars = New_Cars[[col for col in New_Cars.columns if col != 'Car About'] + ['Car About']]

In [146]:
New_Cars.head()

,Model,Model_1,Model Year,Car Status,Car Price,Body Shape,Transmission,Fuel Type,Engine Capacity,Contact,Location,Publishing Date,Car About


In [21]:
from datetime import date
current_date = date.today()
current_date = str(current_date)
#Save DataFrame to CSV file
New_Cars.to_excel(f'N{current_date}.xlsx', index=False)

# Used Cars Scraping 

In [36]:
from selenium.common.exceptions import NoSuchElementException

In [37]:
Used_Cars_For_Sale = {'Model':[], 'Model_1': [], 'Model Year': [],'Car Status': [],'Car Price': [],'Lowest price': [],'Average price': [],'Highest price': [],'Body Shape': [],'Transmission': [],'Fuel Type': [],'Engine Capacity': [],"Car About":[],'Contact':[], 'Mileage':[]}

get item Attributes By xpath for each item

In [38]:
def find_element_or_none(driver, xpath):
    try:
        return driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        return None

lock = threading.Lock()

def get_Used_item_info(driver, url):
    # Scroll to the bottom of the page to load all elements
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height 
    
    # All info
    a = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div/h1/span[1]")

    b = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div/h1/span[2]")

    c = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div/h1/span[3]")

    d = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div/h1/span[4]")

    e = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[2]")

    time.sleep(3)

    e1 = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[3]/app-car-price-chart/div[2]/div[3]/p[2]")

    e2 = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[3]/app-car-price-chart/div[2]/div[2]/p[2]")

    e3 = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[3]/app-car-price-chart/div[2]/div[1]/p[2]")

    f = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[1]/app-spec-item/p[1]")

    g = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[2]/app-spec-item/p[1]")

    h = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[4]/app-spec-item/p[1]")

    i = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[5]/app-spec-item/p[1]")

    j = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[1]/div[1]/div")

    t = find_element_or_none(driver, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[2]/div[1]/div[3]/app-spec-item/p[1]")
    
    from selenium.common.exceptions import NoSuchElementException
    k = None
    con = None
    try:
        k = driver.find_element(By.XPATH, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[4]/a")
        con = k.get_attribute('href')
    except NoSuchElementException:
        try:
            k = driver.find_element(By.XPATH, "/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/app-new-used-car-details/div[1]/contact-car-details-sticky-mobile/div/div[3]/a")
            con = k.get_attribute('href')

        except NoSuchElementException:
            print("Contact button not found.")
            con = None

    with lock:
        Used_Cars_For_Sale['Model'].append(a)
        Used_Cars_For_Sale['Model_1'].append(b)  
        Used_Cars_For_Sale['Model Year'].append(c)
        Used_Cars_For_Sale['Car Status'].append(d)
        Used_Cars_For_Sale['Car Price'].append(e)
        Used_Cars_For_Sale['Lowest price'].append(e1)
        Used_Cars_For_Sale['Average price'].append(e2)
        Used_Cars_For_Sale['Highest price'].append(e3)
        Used_Cars_For_Sale['Body Shape'].append(f)
        Used_Cars_For_Sale['Transmission'].append(g)
        Used_Cars_For_Sale['Fuel Type'].append(h)
        Used_Cars_For_Sale['Engine Capacity'].append(i)
        Used_Cars_For_Sale['Car About'].append(j)
        Used_Cars_For_Sale['Mileage'].append(t)    
        Used_Cars_For_Sale['Contact'].append(con)

Get links to all used items on the site

In [39]:
def Used(src):
    driver = webdriver.Chrome()
    driver.get(src)
    wait = WebDriverWait(driver, 10)  # Increase the wait time here (e.g., 10 seconds)
    time.sleep(2)  # Let the page load for a few seconds
    
    # Scroll to the bottom of the page to load all elements
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height 
    used_item_links = []
    
    # //*[@class="n-engine-card__img"]
    Items_List = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[contains(concat(' ', normalize-space(@class), ' '), ' d-block ') and contains(concat(' ', normalize-space(@class), ' '), ' bg--white ') and contains(concat(' ', normalize-space(@class), ' '), ' car-link ')]")))
    for item in Items_List:
        used_item_links.append(item.get_attribute('href'))
        
    print(f"Loading {len(used_item_links)} items..")
    
    ############
    for i in used_item_links:
        time.sleep(1)
        driver.refresh()
        retries = 2   
        for attempt in range(retries):
            try:
                driver.get(i)   
                get_Used_item_info(driver, i)
                break  #  
            except Exception as e:
                time.sleep(1)
                print(f"Failed attempt {attempt+1}/{retries}")
                
    #############
    driver.quit()

get number of pages used items 

In [29]:
url = "https://www.contactcars.com/en/usedcars?page=1&sortOrder=false&sortBy=createdAt"  
xpath = '/html/body/app-root/app-index/div/div[3]/app-car-listing-base/app-index/div[2]/div/div[2]/div[2]/app-new-paginator/div'
p = get_number_of_pages(url, xpath)
used_pages = int(re.findall(r'\d+', p)[-1])
used_pages

186

In [ ]:
for i in range(1,used_pages+1):
    url = f'https://www.contactcars.com/en/usedcars?page={i}&sortOrder=false&sortBy=createdAt'
    Used(url)
    print(f"(used cars) -Page:  {i} Loaded Successfully")

In [6]:
def USED_Cars_Thread(u_p):
    pages = u_p
    import threading
    num_threads = 4
    # Define the function for each thread
    def scrape_thread(page_range):
        for i in page_range:
            url = f'https://www.contactcars.com/en/usedcars?page={i}&sortOrder=false&sortBy=createdAt'
            Used(url)
            print(f"(used cars) -Page:  {i} Loaded Successfully")

    # Create and start the threads
    threads = []
    pages_per_thread = pages // num_threads  # Integer division to split pages evenly

    for t in range(num_threads):
        start_page = t * pages_per_thread + 1
        end_page = (t + 1) * pages_per_thread + 1
        if t == num_threads - 1:  # Handle remaining pages for the last thread
            end_page = pages + 1
        page_range = range(start_page, end_page)
        thread = threading.Thread(target=scrape_thread, args=(page_range,))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()
    print(threads)

In [ ]:
#USED_Cars_Thread(used_pages)

In [136]:
import pandas as pd
Used_Cars = pd.DataFrame(Used_Cars_For_Sale)

In [137]:
Used_Cars['Location'] = Used_Cars['Car About'].apply(lambda x: x.split('\n')[-2] if x else '')
Used_Cars['Publishing Date'] = Used_Cars['Car About'].apply(lambda x: x.split('\n')[-1] if x else '')
Used_Cars = Used_Cars[[col for col in Used_Cars.columns if col != 'Car About'] + ['Car About']]

In [138]:
Used_Cars.head()

,Model,Model_1,Model Year,Car Status,Car Price,Lowest price,Average price,Highest price,Body Shape,Transmission,Fuel Type,Engine Capacity,Contact,Mileage,Location,Publishing Date,Car About
0,Hyundai,Accent RB,2015,Used,"450,000 EGP","339,365 EGP","452,583 EGP","565,800 EGP",Sedan,Automatic,Gas,1600 CC,tel:+201003609367,"260,000 KM",Alexandria,5 days ago,"Hyundai\nAccent RB\n2015\nUsed\nSmouha\n,\nAle..."
1,Chery,Tiggo,2017,Used,"400,000 EGP","299,115 EGP","398,878 EGP","498,640 EGP",SUV,Automatic,Gas,1600 CC,tel:+201276200018,"140,000 KM",Cairo,2 days ago,"Chery\nTiggo\n2017\nUsed\nManial\n,\nCairo\n2 ..."
2,Fiat,Tipo,2021,Used,"750,000 EGP","524,160 EGP","699,010 EGP","873,860 EGP",Sedan,Automatic,Gas,1600 CC,tel:+201004421991,"95,000 KM",Cairo,2 days ago,Fiat\nTipo\n2021\nUsed\n1.6 A/T H/L Sedan\nMok...
3,Kia,Sportage,2022,Used,"1,500,000 EGP","1,023,880 EGP","1,365,210 EGP","1,706,540 EGP",SUV,Automatic,Gas,1600 CC,tel:+201000051283,"24,000 KM",Giza,2 days ago,Kia\nSportage\n2022\nUsed\n1.6 A/T HL Plus\nEl...
4,Toyota,Corolla,2021,Used,"950,000 EGP","720,170 EGP","960,300 EGP","1,200,430 EGP",Sedan,Automatic,Gas,1600 CC,tel:+201115078837,"26,000 KM",Cairo,6 days ago,Toyota\nCorolla\n2021\nUsed\n1.6 A/T Active Pl...
